# MADE ML advanced course

### student: Shchegolev Oleg

### group: DS-22

### Homework #2, CHGK ranking system, EM-algorithm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pickle

## Data preparation

In [2]:
results = pickle.load(open('chgk/results.pkl', 'rb'))
tournaments = pickle.load(open('chgk/tournaments.pkl', 'rb'))
df_players = pd.DataFrame(pd.read_pickle('chgk/players.pkl')).T.drop('id', axis=1)


In [131]:
df_tour_train = pd.DataFrame([ (k, v['name'], 
       v['type']['id'], 
       v['type']['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2019' ],
                   columns = ['id', 'name', 'type_id', 'type_name'])
df_tour_test = pd.DataFrame([ (k, v['name'], 
       v['type']['id'], 
       v['type']['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2020' ],
                   columns = ['id', 'name', 'type_id', 'type_name'])

In [4]:
results_train = [(tour, res) for tour, res in results.items() if tour in df_tour_train['id'].values]
valid_train_tours = [[(tour, item) for item in res if item.get('mask') is not None] \
       for tour, res in results_train] 
valid_train_tours = [item for item in valid_train_tours if item]

results_test = [(tour, res) for tour, res in results.items() if tour in df_tour_test['id'].values and res]

In [67]:
test_data = []
for tour, teams in results_test:
    data = []
    for team in teams:
        members = [member['player']['id'] for member in team['teamMembers']]
        position = team.get('position')
        if position is None:
            continue
        data.append((members, position))
    test_data.append(data)
    
validation_data = []
for tour, teams in results_train:
    data = []
    for team in teams:
        members = [member['player']['id'] for member in team['teamMembers']]
        position = team.get('position')
        if position is None:
            continue
        data.append((members, position))
    validation_data.append(data)

In [68]:
teams_train = []
masks_train = []
tids = []
train_players = set()
n_pairs_player_question = 0
for i, tournament in enumerate(valid_train_tours):
    teams_train.append([])
    masks_train.append([])
    max_mask_size = max([len(team['mask']) for tid, team in tournament])
    for tid, team in tournament:
        tids.append(tid)
        members = [member['player']['id'] for member in team['teamMembers']]
        mask = team['mask']
        if members and len(mask) == max_mask_size:
            teams_train[i].append(members)
            masks_train[i].append(mask)
            n_pairs_player_question += len(set(members)) * len(mask)
            for member in members:
                train_players.add(member)

In [7]:
one_player_data = []
for i in range(len(teams_train)):
    for j in range(len(masks_train[i][0])):
        question = str(i) + '_' + str(j)
        for k in range(len(teams_train[i])):
            for player in teams_train[i][k]:
                answer = masks_train[i][k][j]
                if answer == 'X':
                    continue
                if answer == '?':
                    answer = '0'
                answer = int(answer)
                one_player_data.append([player, question, answer])

In [8]:
len(one_player_data)

17753544

In [92]:
one_player_data[-10:]

[[184710, '674_215', 0],
 [184711, '674_215', 0],
 [184712, '674_215', 0],
 [201800, '674_215', 0],
 [201801, '674_215', 0],
 [201802, '674_215', 0],
 [207830, '674_215', 0],
 [207831, '674_215', 0],
 [207832, '674_215', 0],
 [210786, '674_215', 0]]

### One hot encoding of pairs (player, question)

In [10]:
np.random.seed(21)
data_train = np.array(one_player_data)
#np.random.shuffle(data_train)
X_train, y_train = data_train[:, :2], data_train[:, 2].astype(bool)

In [13]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
X_train_one_hot = encoder.fit_transform(X_train)

In [14]:
X_train_one_hot.shape

(17753544, 90727)

## Baseline model: logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='saga', random_state=21)
lr.fit(X_train_one_hot, y_train)

LogisticRegression(random_state=21, solver='saga')

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(lr.predict(X_train_one_hot), y_train)

0.7744280240609988

In [17]:
#naive prediction
accuracy_score(y_train * 0, y_train)

0.5629277737447802

In [18]:
pls = encoder.categories_[0].astype(int)
forces = lr.coef_.ravel()[:encoder.categories_[0].shape[0]].astype(float)
p = np.vstack([pls, forces]).T

In [19]:
top20 = p[p[:, 1].argsort()][::-1][:20, 0]

## Top 20 rating of players by baseline model

In [20]:
df_players.loc[top20.astype(int)]

,name,patronymic,surname
27403,Максим,Михайлович,Руссо
4270,Александра,Владимировна,Брутер
28751,Иван,Николаевич,Семушин
27822,Михаил,Владимирович,Савченков
30270,Сергей,Леонидович,Спешков
30152,Артём,Сергеевич,Сорожкин
20691,Станислав,Григорьевич,Мереминский
18036,Михаил,Ильич,Левандовский
26089,Ирина,Сергеевна,Прокофьева
22799,Сергей,Игоревич,Николенко


## Test of baseline model

Firstly I tried to esmimate force of team as mean of it's players:
$$f_t(s_i, i \in t) = \frac{1}{N_{i \in t}}\sum_{i \in t}s_i$$

In [51]:
from scipy.stats import spearmanr as spearman, kendalltau as kendall 


def get_corr_ranking_for_tour(data, p, unknown):
    #unknown players are estimated by a mean value of 'player force'
    unknown_player_estimation = p[:, 1].mean()
    team_powers = []
    real_positions = []
    for team in data:
        if unknown == False:
            for member in team[0]:
                if member not in p[:, 0]:
                    return np.nan, np.nan
        #team power is estimated as mean of players skills
        power = np.mean([p[p[:,0] == member][:,1][0] if member in p[:, 0] \
                                                 else unknown_player_estimation \
                                                 for member in team[0]])
        team_powers.append(power)
        real_positions.append(team[1])
    predicted_positions = np.argsort(np.array(team_powers).ravel())[::-1] + 1
    real_positions = np.argsort(np.array(real_positions).ravel())
    corr1 = spearman(predicted_positions, real_positions)[0]
    corr2 = kendall(predicted_positions, real_positions)[0]
    return corr1, corr2


def get_val_score(val_data, p):
    corrs = []
    for tour in val_data:
        corr12 = get_corr_ranking_for_tour(tour, p, True)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation: {corrs[:, 0].sum() / len(corrs): .2f}')
    print(f'mean kendall correlation: {corrs[:, 1].sum() / len(corrs): .2f}')
    

def get_test_score(test_data, p):
    corrs = []
    for tour in test_data:
        corr12 = get_corr_ranking_for_tour(tour, p, True)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation with unknown players: {corrs[:, 0].mean(): .2f}')
    print(f'mean kendall correlation with unknown players: {corrs[:, 1].mean(): .2f}')
    corrs = []
    for tour in test_data:
        corr12 = get_corr_ranking_for_tour(tour, p, False)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation without unknown players: {corrs[:, 0].mean(): .2f}')
    print(f'mean kendall correlation without unknown players: {corrs[:, 1].mean(): .2f}')
    
    
print('Ranking on train set:\n')
get_val_score(validation_data, p)
print('\n\nRanking on test set:\n')
get_test_score(test_data, p)
    

Ranking on train set:

mean spearman correlation:  0.84
mean kendall correlation:  0.69


Ranking on test set:

mean spearman correlation with unknown players:  0.75
mean kendall correlation with unknown players:  0.58
mean spearman correlation without unknown players:  0.71
mean kendall correlation without unknown players:  0.63


In test set we have some unknown players and I calculated correlations in 2 ways:

1) assigned the mean skill of players from train set to all unknown players 

2) dropped all tournaments with unknown players and calculated only correlations for the rest

### Ranking based on statistical team force estimation (comparing probabilities of right answer for question of zero complexity)

Unfortunately I couldn't come up with better idea about estimation of probability of (team gives right answer) based on probabilities of it's players. So I read about some ranking algorithms in Internet and found paper [1]. So I used estimation from there.

$$p(\exists i \in t: z_{i} = 1) = 1 - \prod_{i \in t}(1 - \sigma(\mu + s_i))$$

Sure it's very logical that probability of (team knows true answer for q) is 1 - production of probabilities that nobody in team knows true answer, but I counld't come up with it by myself(

[1] - Nikolenko, S. (2015). A Probabilistic Rating System for Team Competitions with Individual Contributions. Analysis of Images, Social Networks and Texts, 3–13

In [56]:
def sigmoid(x):
    ex = np.exp(x)
    return ex / (1 + ex)

probs = p.copy()
probs[:, 1] = sigmoid(p[:, 1] + lr.intercept_)

def get_corr_ranking_for_tour2(data, p, unknown):
    #unknown players are estimated by a mean value of 'player force'
    unknown_player_estimation = p[:, 1].mean()
    team_powers = []
    real_positions = []
    for team in data:
        if unknown == False:
            for member in team[0]:
                if member not in p[:, 0]:
                    return np.nan, np.nan
        #team power is estimated as mean of players skills
        power = 1.
        for prob in [p[p[:,0] == member][:,1][0] if member in p[:, 0] \
                                                 else unknown_player_estimation \
                                                 for member in team[0]]:
            power *= (1 - prob)
        power = 1. - power
        team_powers.append(power)
        real_positions.append(team[1])
    predicted_positions = np.argsort(np.array(team_powers).ravel())[::-1] + 1
    real_positions = np.argsort(np.array(real_positions).ravel())
    corr1 = spearman(predicted_positions, real_positions)[0]
    corr2 = kendall(predicted_positions, real_positions)[0]
    return corr1, corr2


def get_val_score2(val_data, p):
    corrs = []
    for tour in val_data:
        corr12 = get_corr_ranking_for_tour2(tour, p, True)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation: {corrs[:, 0].sum() / len(corrs): .2f}')
    print(f'mean kendall correlation: {corrs[:, 1].sum() / len(corrs): .2f}')
    

def get_test_score2(test_data, p):
    corrs = []
    for tour in test_data:
        corr12 = get_corr_ranking_for_tour2(tour, p, True)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation with unknown players: {corrs[:, 0].mean(): .2f}')
    print(f'mean kendall correlation with unknown players: {corrs[:, 1].mean(): .2f}')
    corrs = []
    for tour in test_data:
        corr12 = get_corr_ranking_for_tour2(tour, p, False)
        if corr12[0] is np.nan:
            continue
        corrs.append(corr12)
    corrs = np.array(corrs)
    print(f'mean spearman correlation without unknown players: {corrs[:, 0].mean(): .2f}')
    print(f'mean kendall correlation without unknown players: {corrs[:, 1].mean(): .2f}')
    
print('Ranking on train set:\n')
get_val_score(validation_data, probs)
print('\n\nRanking on test set:\n')
get_test_score(test_data, probs)


Ranking on train set:



/home/oleg/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/oleg/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


mean spearman correlation:  0.84
mean kendall correlation:  0.68


Ranking on test set:

mean spearman correlation with unknown players:  0.74
mean kendall correlation with unknown players:  0.57
mean spearman correlation without unknown players:  0.71
mean kendall correlation without unknown players:  0.63


As one can see ranking correlations are almost equal for both techniques for the baseline algorithm, but the big difference exists for em-algorithm. I found that using the first method (mean value of team players) of ranking I get correlation decrease step by step of em. Using the second method correlation increases a little bit.  

## EM Algrorithm

EM-algorithm was also taken from [1]

Let us assume that we have latent variable: $z_{iq}$ that is if player i knows answer for question q.
So at E-step we will estimate $z_{iq}$ and at M-step learn player force and question complexity. Baseline model weights are used as a start point

As it is written in the paper we assume that $z_{iq}$ is equal to 0 if team gave wrong answer and it is equal probability of i player to know answer for question q divided to probability that at least one player in team knows right answer

[1] Nikolenko, S. (2015). A Probabilistic Rating System for Team Competitions with Individual Contributions. Analysis of Images, Social Networks and Texts, 3–13

### E-step, M-step

### data preparation for denominator calculation

In [22]:
df_train_pls = pd.DataFrame(np.arange(pls.shape[0]), index=pls)

In [23]:
#from tqdm.notebook import tqdm, tnrange
team_ids = []
teams_pls = []
for tour in teams_train:
    for t in tour:
        team = df_train_pls.loc[t].values.ravel()
        teams_pls.append(team)

for i in range(len(teams_train)):
    for j in range(len(masks_train[i][0])):
        question = str(i) + '_' + str(j)
        for k in range(len(teams_train[i])):
            for player in teams_train[i][k]:
                answer = masks_train[i][k][j]
                if answer == 'X':
                    continue
                team_ids.append(i)
team_ids = np.array(team_ids)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs


In [24]:
def get_team_prob(sigmoid, team_ids, team_pls):
    probs = np.ones(team_ids.shape[0])
    i = 0
    while (i < len(team_ids)):
        team = team_pls[team_ids[i]]
        l = len(team)
        team_probs = -(sigmoid[i:i + l] - 1.)
        probs[i: i + l] -= np.prod(team_probs)
        i += l
    return probs


### Implementation of EM-algorithm

In [59]:
from sklearn.linear_model import LinearRegression

def sigmoid(x):
    ex = np.exp(x)
    return ex / (1 + ex)

def e_step(z):
    teams_probs = get_team_prob(z, team_ids, teams_pls)
    teams_probs[z == 0.] = 1.
    return z / teams_probs
        
def m_step(X, z):
    z = np.clip(z, 1e-8, 1 - 1e-8)
    inv_sig_z = np.log(z / (1 - z))
    linr = LinearRegression()
    linr.fit(X, inv_sig_z)
    return linr

def em_algo_with_test(n_steps, X, y, baseline, test_data, players):
    z0 = baseline.predict_proba(X)[:, 1] * y
    for i in range(1, n_steps + 1):
        z = e_step(z0)
        step_model = m_step(X, z)
        forces = step_model.coef_.ravel()[:players.shape[0]].astype(float)
        p = np.vstack([players, forces]).T
        probs = p.copy()
        probs[:, 1] = sigmoid(p[:, 1] + lr.intercept_)
        print(f'-------------step {i}---------------\n')
        print('Ranking on train set:\n')
        get_val_score2(validation_data, probs)
        print('\n\nRanking on test set:\n')
        get_test_score2(test_data, probs)
        print()
        z0 = sigmoid(step_model.predict(X)) * y
    return step_model

### learning few steps

In [60]:
final_model = em_algo_with_test(5, X_train_one_hot, y_train, lr, test_data, pls)

-------------step 1---------------

Ranking on train set:

mean spearman correlation:  0.83
mean kendall correlation:  0.67


Ranking on test set:

mean spearman correlation with unknown players:  0.75
mean kendall correlation with unknown players:  0.58
mean spearman correlation without unknown players:  0.70
mean kendall correlation without unknown players:  0.56

-------------step 2---------------

Ranking on train set:

mean spearman correlation:  0.84
mean kendall correlation:  0.68


Ranking on test set:

mean spearman correlation with unknown players:  0.75
mean kendall correlation with unknown players:  0.59
mean spearman correlation without unknown players:  0.72
mean kendall correlation without unknown players:  0.59

-------------step 3---------------

Ranking on train set:

mean spearman correlation:  0.84
mean kendall correlation:  0.68


Ranking on test set:

mean spearman correlation with unknown players:  0.75
mean kendall correlation with unknown players:  0.59
mean sp

## Tournament ranking on question complexity

Ranking of tournaments by the mean value of question complexity

In [134]:
quests = np.unique(X_train[:, 1])
compls = final_model.coef_.ravel()[encoder.categories_[0].shape[0]:].astype(float)
q = np.vstack([quests, compls]).T

tids = np.unique(np.array(tids))

df_q = pd.DataFrame(q, columns=['qid', 'c'])
df_q['id'] = df_q['qid'].apply(lambda x: tids[int(x.split('_')[0])])
df_q['q_num'] = df_q['qid'].apply(lambda x: int(x.split('_')[1]))

df_tour_train.index = df_tour_train.id

df_q = df_q.join(df_tour_train.drop('id', axis=1), on='id')
df_q.index = df_q['qid']
df_q.drop('qid', axis=1, inplace=True)
df_q['c'] = df_q['c'].astype(float)

rating = df_q.groupby(['id']).mean().sort_values(by='c').drop(['q_num'], axis=1)
rating['name'] = df_tour_train.loc[rating.index]['name']
rating[:20]

,c,type_id,name
id,,,
6149,-9.693906,2.0,Чемпионат Санкт-Петербурга. Первая лига
5928,-6.642565,3.0,Угрюмый Ёрш
5684,-6.036465,3.0,Синхрон высшей лиги Москвы
6101,-5.937266,3.0,Воображаемый музей
5159,-5.818303,3.0,Первенство правого полушария
5941,-5.714677,2.0,Чемпионат Мира. Этап 2. Группа А
5942,-5.525652,2.0,Чемпионат Мира. Этап 2. Группа В
5693,-5.389408,3.0,Знание – Сила VI
5083,-5.327694,3.0,Ускользающая сова


It seems more or less true, most of tournaments are major, there is World Championship in top 20 (finals are at 17 and 19 places) but not at first places...

## Top questions

ranking questions by complexity

In [135]:
df_q.sort_values(by='c')[:10]

,c,id,q_num,name,type_id,type_name
qid,,,,,,
558_14,-14.526322,5948,14,Чемпионат Мира. Финал. Группа А,2,Обычный
555_14,-14.318883,5945,14,Чемпионат Мира. Этап 3. Группа А,2,Обычный
551_13,-14.313498,5941,13,Чемпионат Мира. Этап 2. Группа А,2,Обычный
548_4,-13.380086,5938,4,Чемпионат Мира. Этап 1. Группа А,2,Обычный
551_6,-13.211572,5941,6,Чемпионат Мира. Этап 2. Группа А,2,Обычный
555_23,-13.051173,5945,23,Чемпионат Мира. Этап 3. Группа А,2,Обычный
179_39,-12.287751,5465,39,Чемпионат России,2,Обычный
179_67,-12.287751,5465,67,Чемпионат России,2,Обычный
555_9,-12.260129,5945,9,Чемпионат Мира. Этап 3. Группа А,2,Обычный


The situation here is even better: all first places are taken by World Championship